## **Predicting Links with Traditional Methods**
- Link prediction is the problem of predicting the existence of a link between two nodes.
#### **Heuristic Techniques**
- Heuristic techniques can be divided into two categories - local (1-hop and 2-hop neighbours) and global.
- Local heuristics measure the similarity between two nodes by considering their local neighbourhood.
1. Common neighours counts the number of 1-hop neighbours two nodes have in common.
$$f(u,v) = |N(u) \cap N(v)|$$
2. Jaccard's coefficient measures the proportion of 1-hop neighbours shared by two nodes. It normalizes the result by the total number of neighbours so that it rewards nodes with few interconnected neighbours instead of nodes with high degrees.
$$f(u,v) = \frac{|N(u) \cap N(v)|}{|N(u) \cup N(v)|}$$
3. Adamic-Adar index sums the inverse logarithmic degree of neighbours shared by the two target nodes. The idea is that common neighbours with large neighbourhoods are less significant than those with small neighbourhoods.
$$f(u,v) = \sum_{x \in N(u) \cap N(v)} \frac{1}{log |N(x)|}$$
- Global heuristics consider an entire network instead of a local neighbourhood.
1. Katz index computes the weighted sum of every possible path between two nodes. Weights correspond to a discount factor $\beta \in [0,1]$. Two nodes are more likely to be connected if there are many short paths between them. Note that paths of any length can be calculated using adjacency matrix powers $A^n$.
$$f(u,v) = \sum_{i=1}^{\infty} \beta^i A^i$$
2. Random walk with restart performs random walks, starting from a target node. After each walk, it increases the visit count of the current node. With an $\alpha$ probability, it restarts the walk at the target node. Otherwise, it continues its random walk. After a predefined number of iterations, we stop the algorithm and suggest links between target node and nodes with highest visit counts.

#### **Matrix Factorization**
- Matrix factorization predicts links by predicting the entire adjacency matrix $\hat{A}$. This is performed using node embeddings. Similar nodes $u,v$ should have similar embeddings $z_u, z_v$.
- If two nodes are similar, $z_uz_v^T$ should be maximal. If two nodes are different, $z_uz_v^T$ should be minimal.
$$A_{uv} \approx z_v^Tz_u$$
- The goal is to minimize the L2-norm between $A_{uv}$ and $z_v^Tz_u$.
$$min_z \sum_{i \in V, j \in V} (A_{uv} - z_v^Tz_u)^2$$
- However, matrix factorization cannot use node features and cannot capture structural similarity.

## **Predicting Links with Node Embeddings**
#### **Graph Autoencoders**
- The encoder is a two layer GCN that computes node embeddings. Note that the encoder can be replaced with another type of GNN, for example GraphSAGE.
$$Z = GCN(X, A)$$
- The decoder approximates the adjacency matrix $\hat{A}$ using matrix factorization and a sigmoid function $\sigma$ to output probabilities for each element of the adjacency matrix.
$$\hat{A} = \sigma(Z^TZ)$$
- The GAE is trained using the binary cross-entropy loss between elements of both adjacency matrices.
$$L_{BCE} = \sum_{i \in V, j \in V} -A_{ij} log(\hat{A_{ij}}) - (1-A_{ij})log(1 - \hat{A_{ij}})$$

#### **Graph Variational Autoencoders**
- Instead of directly learning node embeddings, VGAE learns normal distributions that are sampled to produce embeddings.
1. The encoder is composed of two GCNs that share their first layer. The objective is to learn the parameters of each latent normal distributions - a mean $\mu_i$ (learned by $GCN_{mu}$) and a variance $\sigma_i^2$ (learned by $GCN_{\sigma}$).
2. The decoder samples embeddings $z_i$ from the learned distributions $N(\mu_i,\sigma_i^2)$ and $\hat{A} = \sigma(Z^TZ)$.
- Kullback-Leiber (KL) divergence is added to the loss function to ensure that the encoder's output follows a normal distribution. This is known as evidence lower bound (ELBO).
$$L_{ELBO} = L_{BCE} - KL[q(Z|X,A) || p(Z)]$$
where $q(Z|X,A)$ represents the encoder and $p(Z)$ is the prior distribution of $Z$.
- The model's performance is generally evaluated using AU-ROC and average precision.

## **Implementing VGAE**

In [1]:
import torch
!pip install -q torch-scatter~=2.1.0 torch-sparse~=0.6.16 torch-cluster~=1.6.0 torch-spline-conv~=1.2.1 torch-geometric==2.2.0 -f https://data.pyg.org/whl/torch-{torch.__version__}.html

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.8/994.8 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
import numpy as np
np.random.seed(0)
import torch
torch.manual_seed(0)
import matplotlib.pyplot as plt
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True, split_labels=True, add_negative_train_samples=False),
])

dataset = Planetoid('.', name='Cora', transform=transform)

train_data, val_data, test_data = dataset[0]

Processing...
Done!
/usr/local/lib/python3.11/dist-packages/torch_geometric/datasets/planetoid.py:94: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = 

In [3]:
from torch_geometric.nn import GCNConv, VGAE

class Encoder(torch.nn.Module):
  def __init__(self, dim_in, dim_out):
    super().__init__()
    self.conv1 = GCNConv(dim_in, 2 * dim_out)
    self.conv_mu = GCNConv(2 * dim_out, dim_out) # Learns mean
    self.conv_logstd = GCNConv(2 * dim_out, dim_out) # Learn log of standard deviation

  def forward(self, x, edge_index):
    x = self.conv1(x, edge_index).relu()
    return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

In [4]:
model = VGAE(Encoder(dataset.num_features, 16)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train():
  model.train()
  optimizer.zero_grad()
  z = model.encode(train_data.x, train_data.edge_index)
  loss = model.recon_loss(z, train_data.pos_edge_label_index) + (1 / train_data.num_nodes) * model.kl_loss()
  loss.backward()
  optimizer.step()
  return float(loss)

@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    return model.test(z, data.pos_edge_label_index, data.neg_edge_label_index)

for epoch in range(301):
    loss = train()
    val_auc, val_ap = test(test_data)
    if epoch % 50 == 0:
        print(f'Epoch {epoch:>2} | Loss: {loss:.4f} | Val AUC: {val_auc:.4f} | Val AP: {val_ap:.4f}')

test_auc, test_ap = test(test_data)
print(f'Test AUC: {test_auc:.4f} | Test AP {test_ap:.4f}')

Epoch  0 | Loss: 3.4533 | Val AUC: 0.6614 | Val AP: 0.6886
Epoch 50 | Loss: 1.3308 | Val AUC: 0.6665 | Val AP: 0.6830
Epoch 100 | Loss: 1.1945 | Val AUC: 0.7290 | Val AP: 0.7204
Epoch 150 | Loss: 1.0716 | Val AUC: 0.7947 | Val AP: 0.7941
Epoch 200 | Loss: 1.0033 | Val AUC: 0.8444 | Val AP: 0.8421
Epoch 250 | Loss: 0.9740 | Val AUC: 0.8507 | Val AP: 0.8484
Epoch 300 | Loss: 0.9479 | Val AUC: 0.8887 | Val AP: 0.8797
Test AUC: 0.8887 | Test AP 0.8797


In [5]:
z = model.encode(test_data.x, test_data.edge_index)
Ahat = torch.sigmoid(z @ z.T)
Ahat

tensor([[0.8625, 0.7527, 0.8332,  ..., 0.3899, 0.8422, 0.7917],
        [0.7527, 0.8153, 0.8529,  ..., 0.4861, 0.8352, 0.7916],
        [0.8332, 0.8529, 0.8979,  ..., 0.4606, 0.8877, 0.8433],
        ...,
        [0.3899, 0.4861, 0.4606,  ..., 0.6122, 0.4758, 0.4470],
        [0.8422, 0.8352, 0.8877,  ..., 0.4758, 0.8948, 0.8351],
        [0.7917, 0.7916, 0.8433,  ..., 0.4470, 0.8351, 0.7928]],
       device='cuda:0', grad_fn=<SigmoidBackward0>)